In [ ]:
import tensorflow as tf

In [ ]:
%%time
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

In [ ]:
%%time
# Creates a graph.
with tf.device('/cpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

In [ ]:
%%time
mat1 = tf.random_normal([30000,30000],mean=0.0, stddev=1.0, dtype=tf.float32)
mat2 = tf.random_normal([30000,30000],mean=0.0, stddev=1.0, dtype=tf.float32)

mat3 = tf.matmul(mat1, mat2)

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(mat3))

In [ ]:
%%time
with tf.device('/cpu:0'):

    mat1 = tf.random_normal([30000,30000],mean=0.0, stddev=1.0, dtype=tf.float32)
    mat2 = tf.random_normal([30000,30000],mean=0.0, stddev=1.0, dtype=tf.float32)
    mat3 = tf.matmul(mat1, mat2)

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(mat3))